In [1]:
import torch
from torch import nn
prediction_layer = nn.Linear(4096, 130528, bias=False).cuda()


hs = prediction_layer.weight[130527]
h0 = torch.normal(0, 10, [4096]).cuda()
h1 = hs - h0

In [2]:
prediction = torch.argmax(prediction_layer(hs))
print("Expected prediction: ", prediction.item())

Expected prediction:  130527


In [3]:
try:
    import sys
    del sys.modules['perm_llm.glm6b.secure_inference']
    del sys.modules['homomorphic_encryption.bfv']
    print("delete complete!")
except Exception as e:
    print(e)
from perm_llm.glm6b.secure_inference import GLM_PredictionProtocol, GLM_EmbeddingRetrievalProtocol
from perm_llm.common.torch_utils import relative_error

'perm_llm.glm6b.secure_inference'


/home/zf/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from perm_llm.common.communication import Communication, Node, SimulatedCommunication
communication = SimulatedCommunication(["n0", "n1", "n2"])
communication.new_stage("Test")

n0 = Node(communication, "n0")
n1 = Node(communication, "n1")
n2 = Node(communication, "n2")

n0.space.word_embedding = prediction_layer.weight

embedding_protocol = GLM_EmbeddingRetrievalProtocol(n0, n1, n2, 10, device="cuda")
prediction_protocol = GLM_PredictionProtocol(n0, n1, n2, embedding_protocol.name, 10, device="cuda")
embedding_protocol.prepare()
prediction_protocol.prepare()
prediction_protocol.offline_execute()

In [5]:
n0.storage[f"{prediction_protocol.name}:x0"] = h0
n1.storage[f"{prediction_protocol.name}:x1"] = h1
prediction_protocol.online_execute()
print("Computed prediction: ", n1.storage[f"{prediction_protocol.name}:z"])

Computed prediction:  130527
